In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\End_to_End_ml_project_chest_CT_scan\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\End_to_End_ml_project_chest_CT_scan'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/proshanta000/End_to_End_ml_project_chest_CT_scan.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="proshanta000"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d856c7bfcbe6c5c979320b3160b26a5a3e1f4355"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from chestCancerClassifier.constants import*
from chestCancerClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evalution_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/chest_CT_scan_data",
            mlflow_uri="https://dagshub.com/proshanta000/End_to_End_ml_project_chest_CT_scan.mlflow",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE

        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

d:\Data_science\Projects\End_to_End_ml_project_chest_CT_scan\venv\Lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [12]:
class Evalution:
    def __init__(self, config: EvaluationConfig):
        self.config= config


    def _valid_generator(self):
        # ... ( code for setting up generators) ...
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split= 0.20
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        score = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=score)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            
            # CORRECTED: Log each metric individually
            mlflow.log_metric(
                key="loss",
                value=self.score[0]
            )
            mlflow.log_metric(
                key="accuracy",
                value=self.score[1]
            )
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model",
                                       registered_model_name="VGG16Model")
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evalution_config()
    evaluation= Evalution(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-09-22 20:16:32,987: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-22 20:16:32,992: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-22 20:16:32,997: INFO: common: created directory at: artifacts]
Found 178 images belonging to 4 classes.
12/12 [==============================] - 39s 3s/step - loss: 1.2269 - accuracy: 0.6292
[2025-09-22 20:17:12,837: INFO: common: json file saved at: scores.json]


2025/09/22 20:17:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-09-22 20:17:17,544: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\prosh\AppData\Local\Temp\tmpadepnqrm\model\data\model\assets
[2025-09-22 20:17:18,402: INFO: builder_impl: Assets written to: C:\Users\prosh\AppData\Local\Temp\tmpadepnqrm\model\data\model\assets]


d:\Data_science\Projects\End_to_End_ml_project_chest_CT_scan\venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/09/22 20:18:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 6
Created version '6' of model 'VGG16Model'.
